In [8]:
import numpy as np
import time
import sys
        
class Sudoku:
    def __init__(self, matrix):
        self.result  = np.array(matrix)  # 结果表
        self.pending = np.zeros([9,9]).tolist()   # 待定表
        self.history = []                # 回溯表
        
        self._init_pending() # 预生成pending表
        
    def _gen_3x3range(self, pos):
        # 由行/列位置对应3x3九宫格
        # 返回range可用的迭代范围
        rpos = []
        
        if   pos in [0,1,2]:
            rpos.append(0)
            rpos.append(3)
        elif pos in [3,4,5]:
            rpos.append(3)
            rpos.append(6)
        elif pos in [6,7,8]:
            rpos.append(6)
            rpos.append(9)
        
        return rpos
  
    def _check_3x3(self, pos_3x3, num):
        # 检查数字是否在3x3九宫格
        rrow = self._gen_3x3range(pos_3x3[0])
        rcol = self._gen_3x3range(pos_3x3[1])
        
        for row in range(rrow[0], rrow[1]):
            for col in range(rcol[0], rcol[1]):
                if num == self.result[row,col]:
                    return True
            
        return False
    
    def _check_row(self, pos_row, num):
        # 检查数字是否在指定行
        if num not in self.result[pos_row]:
            return False
        else:
            return True
    
    def _check_col(self, pos_col, num):
        # 检查数字是否在指定列
        result_T = self.result.T
        
        if num not in result_T[pos_col]:
            return False
        else:
            return True
    
    def _init_pending(self):
        # 生成每个待解空格的可能解
        print(self.result)
        for row in range(len(self.result)):
            for col in range(len(self.result)):
                if self.result[row][col] == 0:
                    self.pending[row][col] = [[],True] # [[待测试列表],可访问标志]
                    for num in range(1,10):
                        if   self._check_row(row,num):
                            continue
                        elif self._check_col(col,num):
                            continue
                        elif self._check_3x3([row,col],num):
                            continue
                        self.pending[row][col][0].append(num)

    def _get_next(self):
        for row in range(len(self.pending)):
            for col in range(len(self.pending)):
                if self.pending[row][col] != 0:
                    items,v_flag = self.pending[row][col]
                    if (len(items) > 0) and (v_flag == True):
                        num = self.pending[row][col][0].pop() # 弹出顺序找到的一个待处理列表中的元素
                        return [row,col,num,len(self.pending[row][col][0])] # 列表剩余大小处理
                else:
                    continue
        return None
    
    def run(self):
        deep = 0     # 深度标志，从history中恢复数据的条数需从此获得
            
        while True:  # 返回为空即处理完毕
            next_pos = self._get_next() # [行，列，数字，位置空余可选数字个数]
            deep += 1 # 取一个数，加1
            
            if next_pos != None:    
                row, col, num, total = next_pos # 更新当前位置指针
            else:
                return self.result # 数独完成
            
            self.history.append([row,col,num,deep]) # 添加的历史表中待回溯
            if self._check_row(row, num) or self._check_col(col,num) or self._check_3x3([row,col],num): # 冲突
                if total > 0: # 尚有剩余元素，再取一个
                    continue
                else:
                    while True: 
                        for i in range(deep): # 回退当前位置所有元素
                            row, col, num, _ = self.history.pop()
                            self.pending[row][col][0].append(num)
                        
                        if len(self.history) == 0: # 回溯列表为空，说明已回溯到第一个空格，代表数独无解
                            break
                        
                        row, col, _, deep = self.history[-1] # 获得上一位置的行、列以及已压栈数据等信息
                        self.pending[row][col][1] = True     # 下一步继续从上一位置取数
                        self.result[row][col] = 0            # 清除结果表中的相应位置数据
                        
                        if len(self.pending[row][col][0]) == 0: # 如果上一位置待测试数据已空，继续向上回溯
                            continue
                        else:
                            break
                    if len(self.history) == 0:
                        print('数独无解')
                        break
                    else:
                        continue  # 保持deep，防止回退失败
            else:
                self.pending[row][col][1] = False
                self.result[row][col] = num
                deep = 0   # 当前测试值已写入，准备读取下一位置（回退时当前位置所有数据必须全部回退完毕才会读取前一位置信息）
 

In [22]:
sudoku_data = [
    [8,0,0,0,0,0,0,0,0],
    [0,0,3,6,0,0,0,0,0],
    [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0],
    [0,0,0,0,4,5,7,0,0],
    [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8],
    [0,0,8,5,0,0,0,1,0],
    [0,9,0,0,0,0,4,0,0]
]
# sudoku_data = [
#     [0,0,9,0,0,8,0,4,0],
#     [6,0,0,0,0,0,0,1,7],
#     [0,1,0,0,4,0,0,0,0],
#     [0,0,0,0,0,0,0,0,4],
#     [4,8,0,6,0,3,0,2,1],
#     [3,0,0,0,0,0,0,0,0],
#     [0,0,0,0,9,0,0,8,0],
#     [2,4,0,0,0,0,0,0,6],
#     [0,5,0,7,0,0,1,0,0]
# ]
# sudoku_data = [
#     [1,0,5,4,6,2,8,0,0],
#     [0,2,0,1,3,0,0,7,6],
#     [0,0,0,9,7,0,5,0,0],
#     [4,0,2,8,1,0,0,0,0],
#     [7,0,0,3,4,9,0,6,8],
#     [0,3,9,5,0,0,7,4,1],
#     [6,9,0,2,5,0,0,0,0],
#     [0,0,0,0,0,0,0,3,0],
#     [0,8,0,0,0,1,0,2,0]
# ]
start = time.time()
sudoku = Sudoku(sudoku_data)
print(sudoku.run())
print(time.time() - start)

# def reshape_matrix(matrix):
#     sudoku_data = []
#     for i in range(9):
#         sudoku_data.append([ int(i) for i in matrix[i*9:i*9+9] ])
#     return sudoku_data
    
# with open(r'C:\Users\user\Desktop\sudoku17.txt','r') as file:
#      for data in file.readlines():
#         _data = reshape_matrix(data)
#         sudoku = Sudoku(_data)
#         print(sudoku.run())


[[8 0 0 0 0 0 0 0 0]
 [0 0 3 6 0 0 0 0 0]
 [0 7 0 0 9 0 2 0 0]
 [0 5 0 0 0 7 0 0 0]
 [0 0 0 0 4 5 7 0 0]
 [0 0 0 1 0 0 0 3 0]
 [0 0 1 0 0 0 0 6 8]
 [0 0 8 5 0 0 0 1 0]
 [0 9 0 0 0 0 4 0 0]]
[[8 1 2 7 5 3 6 4 9]
 [9 4 3 6 8 2 1 7 5]
 [6 7 5 4 9 1 2 8 3]
 [1 5 4 2 3 7 8 9 6]
 [3 6 9 8 4 5 7 2 1]
 [2 8 7 1 6 9 5 3 4]
 [5 2 1 9 7 4 3 6 8]
 [4 3 8 5 2 6 9 1 7]
 [7 9 6 3 1 8 4 5 2]]
104.82566046714783


In [17]:
class Sudoku1:
    def __init__(self, matrix):
        self.result = [ col for row in matrix for col in row ]
        self.history = []
        self.pending = [ 0 for row in matrix for col in row ]
        self._init_pending()
        
    def _pos2id(self, pos): # position to list index
        return self._rowid(pos[0])[pos[1]]
    def _id2pos(self, index):
        return [index//9,index%9]
    def _row(self, row):
        return self.result[row*9:row*9+9]
    def _rowid(self, row):
        return [ i for i in range(row*9, row*9+9) ]
    def _col(self, col):
        return [ self.result[i*9+col] for i in range(9) ]
    def _colid(self, col):
        return [ i*9+col for i in range(9) ]
    def _blockbypos(self, pos):
        return [ self.result[pos[0]//3*27+pos[1]//3*3+9*br+bc] for br in range(3) for bc in range(3) ]
    def _blockidbypos(self, pos):
        return [ pos[0]//3*27+pos[1]//3*3+9*i+j for i in range(3) for j in range(3) ]
    def _checkbypos(self,pos,num):
        return  True if num in self._row(pos[0]) else \
                True if num in self._col(pos[1]) else \
                True if num in self._blockbypos(pos) else \
                False   
    def _checkbyid(self,index,num):
        return self._checkbypos(self._id2pos(index),num)

    def _init_pending(self):
        for i in range(81):
            if self.result[i] == 0:
                self.pending[i] = [[],True]
                for num in range(1,10):
                    if self._checkbyid(i,num)==False:
                        self.pending[i][0].append(num)
    
    def run(self):
        while True:
            for _index in range(len(self.pending)):
                node = self.pending[_index] # [ [numlist], Status Flag]
                if node != 0: # is list
                    if (len(node[0])>0)and(node[1]==True):
                        break
                node = None
            if node == None:
                return self.result
            
            numlist, s_flag = node
            num = numlist.pop()
            self.history.append([_index,num])
            
            if self._checkbyid(_index,num): # 冲突
                if len(numlist) > 0: # 尚有剩余元素，再取一个
                    continue
                index = _index
                while True: 
                    while (len(self.history)>0) and (self.history[-1][0] == index): # rollback all nums
                        self.pending[index][0].append(self.history.pop()[1])
                        
                    if len(self.history) == 0: # 回溯列表为空，说明已回溯到第一个空格，代表数独无解
                        print('数独无解')
                        return 

                    index, _ = self.history[-1] # 获得上一位置的行、列以及已压栈数据等信息
                    self.pending[index][1] = True     # 下一步继续从上一位置取数
                    self.result[index] = 0            # 清除结果表中的相应位置数据

                    if len(self.pending[index][0]) != 0: # 如果上一位置待测试数据已空，继续向上回溯
                        break
            else:
                self.pending[_index][1] = False
                self.result[_index] = num
                
                
sudoku_data = [
#     # 0 1 2 3 4 5 6 7 8
     [8,0,0,0,0,0,0,0,0], # 0  [00-02]  [03-05]  [06-08]
     [0,0,3,6,0,0,0,0,0], # 1  [09-11]  [12-14]  [15-17]
     [0,7,0,0,9,0,2,0,0], # 2  [18-20]  [21-23]  [24-26]
     [0,5,0,0,0,7,0,0,0], # 3  [27-29]  [30-32]  [33-35]
     [0,0,0,0,4,5,7,0,0], # 4  [36-38]  [39-41]  [42-44]
     [0,0,0,1,0,0,0,3,0], # 5  [45-47]  [48-50]  [51-53]
     [0,0,1,0,0,0,0,6,8], # 6  [54-56]  [57-59]  [60-62]
     [0,0,8,5,0,0,0,1,0], # 7  [63-65]  [66-68]  [69-71]
     [0,9,0,0,0,0,4,0,0]  # 8  [72-74]  [75-77]  [78-80]
#     [1,0,5,4,6,2,8,0,0],
#     [0,2,0,1,3,0,0,7,6],
#     [0,0,0,9,7,0,5,0,0],
#     [4,0,2,8,1,0,0,0,0],
#     [7,0,0,3,4,9,0,6,8],
#     [0,3,9,5,0,0,7,4,1],
#     [6,9,0,2,5,0,0,0,0],
#     [0,0,0,0,0,0,0,3,0],
#     [0,8,0,0,0,1,0,2,0]
]

def reshape_matrix(matrix):
    sudoku_data = []
    for i in range(9):
        sudoku_data.append([ int(i) for i in matrix[i*9:i*9+9] ])
    return sudoku_data

sudoku = Sudoku1(sudoku_data)
for i in reshape_matrix(sudoku.run()):
    print(i)

[8, 1, 2, 7, 5, 3, 6, 4, 9]
[9, 4, 3, 6, 8, 2, 1, 7, 5]
[6, 7, 5, 4, 9, 1, 2, 8, 3]
[1, 5, 4, 2, 3, 7, 8, 9, 6]
[3, 6, 9, 8, 4, 5, 7, 2, 1]
[2, 8, 7, 1, 6, 9, 5, 3, 4]
[5, 2, 1, 9, 7, 4, 3, 6, 8]
[4, 3, 8, 5, 2, 6, 9, 1, 7]
[7, 9, 6, 3, 1, 8, 4, 5, 2]


In [20]:
import time

class Sudoku1:
    def __init__(self, matrix):
        self.result = [ col for row in matrix for col in row ]
        self.history = []
        self.pending = [ 0 for row in matrix for col in row ]
        self._init_pending()
        
    def _checkbyid(self,index,num):
        row = index//9; col = index%9
        return  True if num in self.result[row*9:row*9+9] else \
                True if num in [ self.result[i*9+col] for i in range(9) ] else \
                True if num in [ self.result[row//3*27+col//3*3+9*br+bc] for br in range(3) for bc in range(3) ] else \
                False   

    def _init_pending(self):
        for i in range(81):
            if self.result[i] == 0:
                self.pending[i] = [[],True]
                for num in range(1,10):
                    if self._checkbyid(i,num)==False:
                        self.pending[i][0].append(num)
    
    def run(self):
        while True:
            for _index in range(len(self.pending)):
                node = self.pending[_index] # [ [numlist], Status Flag]
                if node != 0: # is list
                    if (len(node[0])>0)and(node[1]==True):
                        break
                node = None
            if node == None:
                return self.result
            
            numlist, s_flag = node
            num = numlist.pop()
            self.history.append([_index,num])
            
            if self._checkbyid(_index,num): # 冲突
                if len(numlist) > 0: # 尚有剩余元素，再取一个
                    continue
                index = _index
                while True: 
                    while (len(self.history)>0) and (self.history[-1][0] == index): # rollback all nums
                        self.pending[index][0].append(self.history.pop()[1])
                        
                    if len(self.history) == 0: # 回溯列表为空，说明已回溯到第一个空格，代表数独无解
                        print('数独无解')
                        return 

                    index, _ = self.history[-1] # 获得上一位置的行、列以及已压栈数据等信息
                    self.pending[index][1] = True     # 下一步继续从上一位置取数
                    self.result[index] = 0            # 清除结果表中的相应位置数据

                    if len(self.pending[index][0]) != 0: # 如果上一位置待测试数据已空，继续向上回溯
                        break
            else:
                self.pending[_index][1] = False
                self.result[_index] = num
                
                
sudoku_data = [
     [8,0,0,0,0,0,0,0,0],
     [0,0,3,6,0,0,0,0,0],
     [0,7,0,0,9,0,2,0,0],
     [0,5,0,0,0,7,0,0,0],
     [0,0,0,0,4,5,7,0,0],
     [0,0,0,1,0,0,0,3,0],
     [0,0,1,0,0,0,0,6,8],
     [0,0,8,5,0,0,0,1,0],
     [0,9,0,0,0,0,4,0,0] 
]

def reshape_matrix(matrix):
    sudoku_data = []
    for i in range(9):
        sudoku_data.append([ int(i) for i in matrix[i*9:i*9+9] ])
    return sudoku_data

start = time.time()
sudoku = Sudoku1(sudoku_data)
for i in reshape_matrix(sudoku.run()):
    print(i)
print(time.time() - start)

[8, 1, 2, 7, 5, 3, 6, 4, 9]
[9, 4, 3, 6, 8, 2, 1, 7, 5]
[6, 7, 5, 4, 9, 1, 2, 8, 3]
[1, 5, 4, 2, 3, 7, 8, 9, 6]
[3, 6, 9, 8, 4, 5, 7, 2, 1]
[2, 8, 7, 1, 6, 9, 5, 3, 4]
[5, 2, 1, 9, 7, 4, 3, 6, 8]
[4, 3, 8, 5, 2, 6, 9, 1, 7]
[7, 9, 6, 3, 1, 8, 4, 5, 2]
51.593979358673096
